In [1]:
library(edgeR)
library(stringr)

Loading required package: limma



In [2]:
counts = read.csv("data/fshd.counts.matrix.csv")
meta <- read.csv("data/fshd.metadata.csv")

# reformat data frame to be a numerical matrix (same as before)
mat = counts[,c(3:ncol(counts))] 
rownames(mat) = counts$Gene_ID

In [3]:
# custom function
glm_test = function(mat, meta, comparison){

# Make groups to compare
group <- factor(str_sub(meta$Sample, end=-6))

# read count matrix into DGEList
y=DGEList(counts=mat, group=group)

# Filter lowly expressed genes
keep <- filterByExpr(y)
y <- y[keep, , keep.lib.sizes=FALSE]
y <- calcNormFactors(y)

# indicate reps with design matrix
design = model.matrix(~0+group, data=y$samples)
colnames(design) = levels(y$samples$group)

# fit data to generalized linear model
y=estimateCommonDisp(y)
y=estimateTagwiseDisp(y)
fit = glmQLFit(y, design)

# differential expression test
Contrast <- makeContrasts(contrasts= comparison, levels=design)
qlf <- glmQLFTest(fit, contrast=Contrast)

# Get results
table = qlf$table
table$Gene_ID = rownames(table)
return(table)
}

In [4]:
table = glm_test(mat,meta,"Control_78_Day_3-Control_78_Day_0")


In [5]:
write.csv(table,file="results/control_day0_day3_de_results.csv",row.names=F)